# Claim Approval Process


In [1]:
# To ignore any warnings
import warnings
warnings.filterwarnings('ignore')
#To run all the commands in a same shell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from matplotlib import pyplot
import numpy as np
import pandas as pd
from scipy.stats import norm
# For plotting graphs
import matplotlib.pyplot as plt

%matplotlib inline
# For data visualization
import seaborn as sns
#for different algorithms
from sklearn.utils import shuffle
from factor_analyzer import FactorAnalyzer
from sklearn import decomposition, preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

# Import Dataset

In [2]:
df= pd.read_csv((r'C:/Users/Pavan/Documents/DATAEFG/TrainingData_New.csv'))


In [3]:
df.Claimstatus=df.Claimstatus.map({'C':1,'D':0}) 
df.isnull().sum()

ClaimID                 0
AGMID                   0
LOB                     0
State             1604742
ProgramCode             0
PlanCode                0
VehicleMake           123
VehicleModel         3829
VehicleYear             0
TermMonth               0
TermMiles               0
ComponentCode           0
ComponentSet           82
ContractStatus          0
AgeAtClaim              2
AgeAtCancel             0
Claimstatus       2505572
ClaimDate             382
PaidDate                0
AgeofPayment            0
dtype: int64

In [4]:
df.drop_duplicates(['ClaimID', 'ComponentSet'],keep= 'last', inplace = True)


In [6]:
#df.to_csv('C:/Users/Pavan/Documents/DATAEFG/Out_RESULT/T8020.csv',index=False)

In [7]:
df.columns

Index(['ClaimID', 'AGMID', 'LOB', 'State', 'ProgramCode', 'PlanCode',
       'VehicleMake', 'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
       'ComponentCode', 'ComponentSet', 'ContractStatus', 'AgeAtClaim',
       'AgeAtCancel', 'Claimstatus', 'ClaimDate', 'PaidDate', 'AgeofPayment'],
      dtype='object')

In [8]:
nulls = df.isnull().sum()
nulls[nulls > 0]

State           1079158
VehicleMake          93
VehicleModel       3097
ComponentSet         65
AgeAtClaim            1
Claimstatus     1706189
ClaimDate           232
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1706189 entries, 0 to 2505571
Data columns (total 20 columns):
ClaimID           int64
AGMID             int64
LOB               object
State             object
ProgramCode       object
PlanCode          object
VehicleMake       object
VehicleModel      object
VehicleYear       int64
TermMonth         int64
TermMiles         int64
ComponentCode     object
ComponentSet      object
ContractStatus    object
AgeAtClaim        float64
AgeAtCancel       int64
Claimstatus       float64
ClaimDate         float64
PaidDate          int64
AgeofPayment      object
dtypes: float64(3), int64(7), object(10)
memory usage: 273.4+ MB


In [10]:
#df.to_csv('C:/Users/Pavan/Documents/DATAEFG/Out_RESULT/traint_NEW.csv',index=False)

In [11]:
df.shape

(1706189, 20)

In [12]:
from sklearn.utils import shuffle
df = shuffle(df)

In [13]:
df.columns,df.shape

(Index(['ClaimID', 'AGMID', 'LOB', 'State', 'ProgramCode', 'PlanCode',
        'VehicleMake', 'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
        'ComponentCode', 'ComponentSet', 'ContractStatus', 'AgeAtClaim',
        'AgeAtCancel', 'Claimstatus', 'ClaimDate', 'PaidDate', 'AgeofPayment'],
       dtype='object'), (1706189, 20))

In [14]:
#df.to_csv('C:/Users/Pavan/Documents/DATAEFG/Out_RESULT/qwe.csv',index=False)

# Missing values at individual variable level

In [15]:
#test[['ClaimID','ClaimStatus']].to_csv('C:/Users/Pavan/Documents/DATAEFG/Out_RESULT/Test_NEW.csv',index=False)

In [16]:
#shows the missing values at individual variable level

#df.isnull().sum() --> works same as below code
df.apply(lambda x: sum(x.isnull()), axis=0)

ClaimID                 0
AGMID                   0
LOB                     0
State             1079158
ProgramCode             0
PlanCode                0
VehicleMake            93
VehicleModel         3097
VehicleYear             0
TermMonth               0
TermMiles               0
ComponentCode           0
ComponentSet           65
ContractStatus          0
AgeAtClaim              1
AgeAtCancel             0
Claimstatus       1706189
ClaimDate             232
PaidDate                0
AgeofPayment            0
dtype: int64

In [17]:
#shows the total number of missing values
df.isnull().sum().sum()

2788835

In [18]:
#variables which we can remove
var_null_pc = df.isnull().sum(axis=0).sort_values( ascending=False)/float(len(df) )
var_null_pc[ var_null_pc > 0.75 ]

Claimstatus    1.0
dtype: float64

In [19]:
df.drop( var_null_pc[ var_null_pc > 0.75 ].index, axis = 1,  ) 
df.dropna( axis = 0, thresh = 30, )

,ClaimID,AGMID,LOB,State,ProgramCode,PlanCode,VehicleMake,VehicleModel,VehicleYear,TermMonth,TermMiles,ComponentCode,ComponentSet,ContractStatus,AgeAtClaim,AgeAtCancel,ClaimDate,PaidDate,AgeofPayment
1396106,92952726,3317477,Call Center,NaN,TCH01,TCULTN,JEEP,COMMANDER,2006,60,100000,AT070,AT,EXPIRED,18.0,0,20130401.0,0,C
1938599,93259225,4541493,Call Center,NaN,TCH11,TCADVCPU,CHEV,TRAILBLAZER LS/LT,2006,60,100000,CF002,CF,CANCELLED,3.0,4,20150402.0,0,D
1491645,92968521,3700390,Call Center,NaN,TCH02,TCELTULT,BUIC,ENCLAVE CXL,2008,48,100000,FW611,FW,EXPIRED,6.0,0,20130513.0,0,C
1792238,93379577,3994029,Agency Services,NaN,SAM01,SAM3000,NISS,PATHFINDER S/SV/SL/PLATNU,2014,36,36000,LOFL2,MAINT,EXPIRED,23.0,0,20150826.0,0,C
681135,730873,1076976,Dealer Services,KS,MAP01,00SUPN,CHEV,01 CHEVROLET K1500 SILVER,2001,60,90000,AT010,AT,EXPIRED,5.0,0,20040826.0,0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996309,932700,1457480,Agency Services,TX,KK81,DZULTN,LEXS,RX 330,2004,36,9999999,DZ100,DZ,EXPIRED,14.0,0,20070622.0,0,C
805738,851319,1172305,Dealer Services,OR,MAP16,SUPRMU,NISS,98 NISSAN QUEST XE/GXE/GL,1998,24,24000,AT614,AT,EXPIRED,21.0,0,20060707.0,0,C
823966,92618705,2224905,Agency Services,NaN,MAP15,HMPLTU,DODG,RAM 3500 QUAD ST/SLT,2006,48,50000,IN100,IN,EXPIRED,8.0,0,20100514.0,0,C
2485636,94307314,5903870,Agency Services,NaN,SPB31,DTWRKB,TOYT,TUNDRA CREWMAX 1794 CREWM,2018,36,9999999,IN100,IN,INFORCE,20.0,0,20191025.0,0,C


,ClaimID,AGMID,LOB,State,ProgramCode,PlanCode,VehicleMake,VehicleModel,VehicleYear,TermMonth,TermMiles,ComponentCode,ComponentSet,ContractStatus,AgeAtClaim,AgeAtCancel,Claimstatus,ClaimDate,PaidDate,AgeofPayment


# COLUMNS REMOVED

In [20]:
#The Variables which can be removed are -,
vars_to_be_removed = [ 'ClaimDate', 'PaidDate', 'AGMID',  'State','AgeofPayment','AgeAtCancel',
 'AgeAtClaim','ComponentCode'
 ]

df.drop(vars_to_be_removed , axis = 1, inplace=True )#inplace=True

In [21]:
#shows the variable names present in the dataframe
list(df.columns.values)

['ClaimID',
 'LOB',
 'ProgramCode',
 'PlanCode',
 'VehicleMake',
 'VehicleModel',
 'VehicleYear',
 'TermMonth',
 'TermMiles',
 'ComponentSet',
 'ContractStatus',
 'Claimstatus']

In [22]:
df.columns

Index(['ClaimID', 'LOB', 'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles', 'ComponentSet',
       'ContractStatus', 'Claimstatus'],
      dtype='object')

In [23]:
# find categorical variables
categorical = [var for var in df.columns if df[var].dtype=='O']
print('There are {} categorical variables'.format(len(categorical)))

There are 7 categorical variables


In [24]:

# find numerical variables
numerical = [var for var in df.columns if df[var].dtype!='O']
print('There are {} numerical variables'.format(len(numerical)))

There are 5 numerical variables


In [25]:
df[categorical].columns

Index(['LOB', 'ProgramCode', 'PlanCode', 'VehicleMake', 'VehicleModel',
       'ComponentSet', 'ContractStatus'],
      dtype='object')

In [26]:
df[numerical].columns

Index(['ClaimID', 'VehicleYear', 'TermMonth', 'TermMiles', 'Claimstatus'], dtype='object')

In [27]:
df.shape

(1706189, 12)

# One hot encoding and dummy encoding on Categorical

In [28]:
#Changing the datatype of the variable in a dataframe
df['ProgramCode'] = df['ProgramCode'].astype('category')
#df['ProductCode'] = df['ProductCode'].astype('category')
df['PlanCode'] = df['PlanCode'].astype('category')

#df['State'] = df['State'].astype('category')
df['ContractStatus'] = df['ContractStatus'].astype('category')
df['ComponentSet'] = df['ComponentSet'].astype('category')
df['LOB'] = df['LOB'].astype('category')
df['VehicleMake'] = df['VehicleMake'].astype('category')
df['VehicleModel'] = df['VehicleModel'].astype('category')
#df['FuelType'] = df['FuelType'].astype('category') 
#df['ComponentCode'] = df['ComponentCode'].astype('category')

In [29]:
# Convert all non-numeric values to number
cat=['LOB',  'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'ContractStatus']

In [30]:
from sklearn import preprocessing
le =preprocessing.LabelEncoder()
for var in cat:
    le = preprocessing.LabelEncoder()
    df[var]=le.fit_transform(df[var].astype('str'))
df.dtypes

ClaimID              int64
LOB                  int32
ProgramCode          int32
PlanCode             int32
VehicleMake          int32
VehicleModel         int32
VehicleYear          int64
TermMonth            int64
TermMiles            int64
ComponentSet      category
ContractStatus       int32
Claimstatus        float64
dtype: object

In [31]:
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder

In [32]:
df.isnull().sum()

ClaimID                 0
LOB                     0
ProgramCode             0
PlanCode                0
VehicleMake             0
VehicleModel            0
VehicleYear             0
TermMonth               0
TermMiles               0
ComponentSet           65
ContractStatus          0
Claimstatus       1706189
dtype: int64

In [33]:

# outlies in discrete variables
for var in [  'VehicleYear', 'TermMonth', 'TermMiles',    
       ]:
    print(df[var].value_counts() / np.float(len(df)))
    print()

2006    7.500459e-02
2007    6.789693e-02
2005    6.587195e-02
2004    6.061286e-02
2008    6.008068e-02
2003    5.154001e-02
1999    4.630905e-02
2000    4.525231e-02
2002    4.426649e-02
2001    4.367629e-02
2011    4.185058e-02
2013    3.919554e-02
2012    3.867274e-02
2015    3.519012e-02
2014    3.417793e-02
2009    3.387843e-02
1998    3.375300e-02
2010    3.295766e-02
1997    3.171044e-02
2016    2.769623e-02
1996    2.222028e-02
2017    2.111548e-02
2018    1.456286e-02
1995    1.430381e-02
2019    6.331655e-03
1994    6.290042e-03
1993    2.766985e-03
1992    1.276529e-03
1991    5.626575e-04
2020    3.879992e-04
1990    2.977396e-04
1989    1.008095e-04
0       7.150439e-05
1988    4.337151e-05
1987    3.164948e-05
1986    1.992745e-05
1985    8.205422e-06
1983    6.447117e-06
1981    2.930508e-06
1982    1.758305e-06
1966    1.758305e-06
1979    1.172203e-06
1984    5.861015e-07
Name: VehicleYear, dtype: float64

60     3.160418e-01
48     1.993830e-01
36     1.850188e-01
72

In [34]:
for var in categorical:
    print(var, ' contains ', len(df[var].unique()), ' labels')

LOB  contains  3  labels
ProgramCode  contains  282  labels
PlanCode  contains  738  labels
VehicleMake  contains  139  labels
VehicleModel  contains  11259  labels
ComponentSet  contains  76  labels
ContractStatus  contains  10  labels


In [35]:
df2=pd.concat([df, pd.get_dummies(df.ComponentSet)], 1).groupby(['ClaimID']).sum().reset_index()
df2

,ClaimID,LOB,ProgramCode,PlanCode,VehicleMake,VehicleModel,VehicleYear,TermMonth,TermMiles,ContractStatus,...,TR,TW,VAS,WACRITVR,WAEXPENSE,WAIVER,WAPR,WAVR,WH,WS
0,167557,4,150,50,62,7806,3984,168,200000,8,...,0,0,0,0,0,0,0,0,0,0
1,172378,0,75,25,17,4128,1993,84,100000,4,...,0,0,0,0,0,0,0,0,0,0
2,173734,2,75,25,119,4381,1993,84,100000,4,...,0,0,0,0,0,0,0,0,0,0
3,175214,4,168,50,84,8472,3986,168,200000,8,...,0,0,0,0,0,0,0,0,0,0
4,176126,4,150,50,30,8252,3986,168,200000,8,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060680,94720355,2,35,188,93,10079,2018,75,9999999,4,...,0,0,0,0,0,0,0,0,0,0
1060681,94720423,2,34,190,89,9627,2016,72,9999999,4,...,0,0,0,0,0,0,0,0,0,0
1060682,94720528,2,35,188,52,10267,2018,72,9999999,4,...,0,0,0,0,0,0,0,0,0,0
1060683,94720662,2,35,188,17,8014,2017,72,9999999,4,...,0,0,0,0,0,0,0,0,0,0


In [36]:
df2.drop([ 'LOB', 'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
       'ContractStatus', 'Claimstatus'], axis=1, inplace=True) 

In [37]:
df1=df.groupby(['ClaimID']).last()
df1

,LOB,ProgramCode,PlanCode,VehicleMake,VehicleModel,VehicleYear,TermMonth,TermMiles,ComponentSet,ContractStatus,Claimstatus
ClaimID,,,,,,,,,,,
167557,2,75,25,31,3903,1992,84,100000,TC,4,NaN
172378,0,75,25,17,4128,1993,84,100000,ST,4,NaN
173734,2,75,25,119,4381,1993,84,100000,EG,4,NaN
175214,2,84,25,42,4236,1993,84,100000,AT,4,NaN
176126,2,75,25,15,4126,1993,84,100000,EC,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...
94720355,2,35,188,93,10079,2018,75,9999999,GAP,4,NaN
94720423,2,34,190,89,9627,2016,72,9999999,GAP,4,NaN
94720528,2,35,188,52,10267,2018,72,9999999,GAP,4,NaN


In [38]:
df1.drop([ 'ComponentSet'], axis=1, inplace=True) 


In [39]:
df1.columns

Index(['LOB', 'ProgramCode', 'PlanCode', 'VehicleMake', 'VehicleModel',
       'VehicleYear', 'TermMonth', 'TermMiles', 'ContractStatus',
       'Claimstatus'],
      dtype='object')

In [41]:
df3=df1.merge(df2, left_on='ClaimID', right_on='ClaimID')
df3.columns

Index(['ClaimID', 'LOB', 'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
       'ContractStatus', 'Claimstatus', 'AC', 'ACCT', 'ADS', 'AT', 'AX', 'BK',
       'BM', 'CAC', 'CB', 'CD', 'CF', 'CH', 'CM', 'CNG', 'CW', 'DC', 'DI',
       'DZ', 'EC', 'ED', 'EG', 'EL', 'EM', 'FE', 'FR', 'FS', 'FW', 'GA', 'GAP',
       'GDO', 'HS', 'HT', 'IM', 'IN', 'KK', 'KR', 'LD', 'LF', 'MAINT', 'MN',
       'MO', 'MT', 'NMC', 'OCR', 'OT', 'PLM', 'PR', 'PS', 'RA', 'RAM', 'RD',
       'RF', 'RR', 'RW', 'SH', 'SHORT', 'SP', 'SS', 'ST', 'SU', 'SUPPLIES',
       'TB', 'TC', 'THEFT', 'TI', 'TR', 'TW', 'VAS', 'WACRITVR', 'WAEXPENSE',
       'WAIVER', 'WAPR', 'WAVR', 'WH', 'WS'],
      dtype='object')

In [44]:
nulls = df.isnull().sum()
nulls[nulls > 0]

ComponentSet         65
Claimstatus     1706189
dtype: int64

In [42]:
df3.shape

(1060685, 86)

In [43]:
df3.to_csv('C:/Users/Pavan/Documents/DATAEFG/EFGTT/New_train_datav3.csv',index=False)

In [44]:
#The Variables which can be removed are -, removed Catogerical  ComponentSet_OTHER column to avoid dummy variable trap
#vars_to_be_removed = [ 'ComponentSet']

#df.drop(vars_to_be_removed , axis = 1, inplace=True )#inplace=True

# END TRAIN DATA CleanUP